In [113]:
import json
import requests as rq
import pandas as pd
import numpy as np
import time
import itertools
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
G=nx.Graph()
import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client.lol_match_db
collection = db.match_list_2016_5x5

list_summoners = pd.read_excel('LVPSummoner.xlsx',
                      sheetname = 'table_output',
                      header = 0,
                      index_col = 0,
                      parse_cols = "A",
                      convert_float = True)
len(list_summoners)

C:\ProgramData\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
C:\ProgramData\Anaconda\envs\py36\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


439

In [114]:
list_properties = []

ego_edge_weights = {}
#la idea de este es luego ordenarlo y acumularlo para graficarlo.
#for s in range(5):
    
for s in range(len(list_summoners)):
    summoner = db.match_list_2016_5x5.find({"_id": str(list_summoners.index[s])})
    for document in summoner:
        
        #añadimos al EGO y lo marcamos
        total_partidas = len(document['matches']['matches'])
        H=nx.Graph(total_matches = total_partidas, ego_id = str(list_summoners.index[s]))
        H.add_node(str(list_summoners.index[s]), ego = 1)
        
        #repaso todas las partidas de un summoner
        
        for match in document['matches']['matches']:
                        
            partida = db.match_details_2016.find({"_id": str(match['matchId'])})       
            
            for document2 in partida:
                #el primer equipo, el 100, tiene los jugadores de 0 a 4. El otro de 5 a 9.
                firstTeamWin = document2['match_detail']['teams'][0]['winner']
                for participant in document2['match_detail']['participantIdentities']:
                    if str(participant['player']['summonerId']) == str(list_summoners.index[s]):
                        if (participant['participantId'] <= 5 and firstTeamWin):

                            team = document2['match_detail']['participantIdentities'][0:5]
                            for a, b in itertools.combinations(team, 2):                        

                                if H.has_edge(str(a['player']['summonerId']), 
                                              str(b['player']['summonerId'])):
                                    # we added this one before, just increase the weight by one
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['weight'] += 1.0/total_partidas
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['matches_together'] += 1
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['win']+= 1.0/total_partidas

                                else:
                                    # new edge. add with weight=1
                                    H.add_edge(str(a['player']['summonerId']), 
                                               str(b['player']['summonerId']), 
                                              weight= 1.0/total_partidas,
                                                matches_together = 1,
                                                win= 1.0/total_partidas)

                        elif (participant['participantId'] <= 5 and not(firstTeamWin)):

                            team = document2['match_detail']['participantIdentities'][0:5]
                            for a, b in itertools.combinations(team, 2):                        

                                if H.has_edge(str(a['player']['summonerId']), 
                                              str(b['player']['summonerId'])):
                                    # we added this one before, just increase the weight by one
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['weight'] += 1.0/total_partidas
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['matches_together'] += 1

                                else:
                                    # new edge. add with weight=1
                                    H.add_edge(str(a['player']['summonerId']), 
                                               str(b['player']['summonerId']), 
                                               weight= 1.0/total_partidas,
                                                matches_together = 1,
                                                win=0.0)

                        elif (participant['participantId'] > 5 and not(firstTeamWin)):

                            team = document2['match_detail']['participantIdentities'][5:10]
                            for a, b in itertools.combinations(team, 2):                        

                                if H.has_edge(str(a['player']['summonerId']), 
                                              str(b['player']['summonerId'])):
                                    # we added this one before, just increase the weight by one
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['weight'] += 1.0/total_partidas
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['matches_together'] += 1
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['win']+= 1.0/total_partidas

                                else:
                                    # new edge. add with weight=1
                                    H.add_edge(str(a['player']['summonerId']), 
                                               str(b['player']['summonerId']), 
                                               weight= 1.0/total_partidas,
                                               matches_together = 1,
                                                win=1.0/total_partidas)
                        else:

                            team = document2['match_detail']['participantIdentities'][5:10]
                            for a, b in itertools.combinations(team, 2):                        

                                if H.has_edge(str(a['player']['summonerId']), 
                                              str(b['player']['summonerId'])):
                                    # we added this one before, just increase the weight by one
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['weight'] += 1.0/total_partidas
                                    H[str(a['player']['summonerId'])][str(b['player']['summonerId'])]['matches_together'] += 1

                                else:
                                    # new edge. add with weight=1
                                    H.add_edge(str(a['player']['summonerId']), 
                                               str(b['player']['summonerId']), 
                                               weight= 1.0/total_partidas,
                                                matches_together = 1,
                                                win=0.0)
                                    
    nx.write_graphml(H, "networks/playerNetwork_"+ str(list_summoners.index[s]) +".graphml")
    
    egonet_census = nx.triadic_census(nx.to_directed(H))
    
    weight_array = []
    for u,v,a in H.edges(H.graph['ego_id'], data=True):
        weight_array.append(a['weight'])
    
    ego_edge_weights[str(list_summoners.index[s])] = weight_array
    
    list_properties.append((str(list_summoners.index[s]),
                            total_partidas, 
                            len(list(H.nodes())), 
                            nx.density(H), 
                            egonet_census['003'], 
                            egonet_census['102'], 
                            egonet_census['201'], 
                            egonet_census['300'], 
                            nx.number_of_cliques(H, H.graph['ego_id']), 
                            nx.node_clique_number(H, H.graph['ego_id']), 
                            np.average(weight_array)))
    
    H.clear()
    
# creo el dataframe
labels = ['ego', 'total_matches', 'nodes', 'density', 
          'triad_003', 'triad_102', 'triad_201', 'triad_300', 
          'num_maximal_cliques', 'max_clique_size', 'avg_weight']
egonet_data = pd.DataFrame.from_records(list_properties, columns=labels)

In [112]:
document

{'_id': '35528908',
 'matches': {'endIndex': 0, 'startIndex': 0, 'totalGames': 0}}

In [115]:
egonet_data.head()

,ego,total_matches,nodes,density,triad_003,triad_102,triad_201,triad_300,num_maximal_cliques,max_clique_size,avg_weight
0,19459077,69,18,0.457516,197,245,247,127,10,7,0.235294
1,20501992,7,22,0.264069,698,532,242,68,7,5,0.190476
2,23407188,206,673,0.008270,49581762,738311,255210,2013,206,5,0.005952
3,27281117,37,45,0.150505,9831,2508,1654,197,23,6,0.090909
4,28306135,111,329,0.017625,5633176,186129,60849,1050,108,5,0.012195


In [116]:
egonet_data.to_csv('egonets.csv', index=False)

In [29]:
H2 = nx.read_graphml("playerNetwork_19459077.graphml")

In [9]:
H2.nodes.data()

NodeDataView({'19459077': {'ego': 1}, '20116736': {}, '28557346': {}, '37859449': {}, '28517552': {}, '37882045': {}, '22344969': {}, '33909732': {}, '20110852': {}, '23214164': {}, '31409116': {}, '19891004': {}, '33571789': {}, '42055706': {}, '25289352': {}, '75357637': {}, '21876725': {}, '43127201': {}})

In [87]:
nx.number_of_cliques(H2, '19459077')

TypeError: object of type 'int' has no len()

443

In [20]:
H3 = nx.read_graphml("playerNetwork_20501992.graphml")
H3.nodes.data()

NodeDataView({'20501992': {'ego': 1}, '61521191': {}, '53037561': {}, '31113043': {}, '35773403': {}, '52662940': {}, '37945613': {}, '52493865': {}, '24658882': {}, '63986793': {}, '22003918': {}, '182419': {}, '40638373': {}, '25914000': {}, '24818653': {}, '21137760': {}, '50766893': {}, '37742947': {}, '44929492': {}, '37158550': {}, '33011554': {}, '52395740': {}})

In [16]:
list(nx.find_cliques(H3))

[['20501992', '44929492', '52395740', '37158550', '33011554'],
 ['20501992', '22003918', '182419', '40638373', '63986793'],
 ['20501992', '31113043', '53037561', '61521191', '35773403'],
 ['20501992', '31113043', '53037561', '24658882', '52493865'],
 ['20501992', '31113043', '53037561', '37945613', '52662940'],
 ['20501992', '31113043', '37742947', '182419', '50766893'],
 ['20501992', '25914000', '182419', '24818653', '21137760']]

In [24]:
#number of cliques busca cliques MAXIMALES así que siempre encontrará el número de equipos distintos que ha formado el jugador
nx.triangles(H3, H3.graph['ego_id'])/H3.graph['total_matches']

5.714285714285714

In [28]:
nx.cliques_containing_node(H3, H3.graph['ego_id'])

[['20501992', '44929492', '52395740', '37158550', '33011554'],
 ['20501992', '22003918', '182419', '40638373', '63986793'],
 ['20501992', '31113043', '53037561', '61521191', '35773403'],
 ['20501992', '31113043', '53037561', '24658882', '52493865'],
 ['20501992', '31113043', '53037561', '37945613', '52662940'],
 ['20501992', '31113043', '37742947', '182419', '50766893'],
 ['20501992', '25914000', '182419', '24818653', '21137760']]

In [31]:
nx.number_of_cliques(H2, H2.graph['ego_id'])
# son como los "grupos distintos" de gente con la que juega, no exactamente los equipos

10

In [32]:
nx.node_clique_number(H2, H2.graph['ego_id'])

7

In [33]:
nx.node_clique_number(H3, H3.graph['ego_id'])

5

In [38]:
print(nx.average_neighbor_degree(H2, nodes= [H2.graph['ego_id']]))
print(nx.average_neighbor_degree(H3, nodes= [H3.graph['ego_id']]))

{'19459077': 7.235294117647059}
{'20501992': 4.809523809523809}


In [61]:
weight_array = []

for u,v,a in H3.edges(H3.graph['ego_id'], data=True):
    
    weight_array.append(a['weight'])
    
print(np.average(weight_array))

0.190476190476


In [80]:
total = 0
for key, value in nx.triangles(H2).items():
    total += value

print(total/3)

127.0


In [83]:
len(list(H2.nodes()))

18

In [117]:
test = pd.read_csv('egonets.csv')

In [118]:
test.head()

,ego,total_matches,nodes,density,triad_003,triad_102,triad_201,triad_300,num_maximal_cliques,max_clique_size,avg_weight
0,19459077,69,18,0.457516,197,245,247,127,10,7,0.235294
1,20501992,7,22,0.264069,698,532,242,68,7,5,0.190476
2,23407188,206,673,0.008270,49581762,738311,255210,2013,206,5,0.005952
3,27281117,37,45,0.150505,9831,2508,1654,197,23,6,0.090909
4,28306135,111,329,0.017625,5633176,186129,60849,1050,108,5,0.012195
